### Imports

In [ ]:
import pandas as pd
import datetime
import os
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
import warnings
from pandas.errors import SettingWithCopyWarning


warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

### Parameters

In [ ]:
buy_threshold = 1.04
sell_threshold = 1.01
starting_balance = 1000000
price_cap_lower = 0
ttm_cap_lower = 0

run_optimizing = True
google_colab = True

### Data loading

In [ ]:
if google_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "111QpHwxMy28NV8I23Bj_ZD1H32H6n7bV"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('11.05 1 mnd test sett full model run.csv')  
    df = pd.read_csv('11.05 1 mnd test sett full model run.csv')
else:
    # Set the path to the root directory
    path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
    # Read dataframes using Dask
    df = pd.read_csv(path + '/data/predictions/11.05 1 mnd test sett full model run.csv')

### Data processing

In [ ]:
# Dates
df['Expiry_date'] = pd.to_datetime(df['Quote_date']) + pd.to_timedelta(df['TTM'] * 365, unit='D')
df['Quote_date'] = pd.to_datetime(df['Quote_date']).dt.date 
df['Expiry_date'] = pd.to_datetime(df['Expiry_date']).dt.date 

# Adding option ID
df["Option_ID"] = df["Expiry_date"].astype(str) + "-" + df["Strike"].astype(str)

df_original = df[(df["Quote_date"] >= pd.to_datetime("2018-01-01").date()) & 
                 (df["Quote_date"] <= pd.to_datetime("2019-01-01").date())]

df = df[(df["Expiry_date"] >= pd.to_datetime("2018-01-01").date()) & 
        (df["Expiry_date"] <= pd.to_datetime("2018-12-31").date())]

df = df[(df["Quote_date"] >= pd.to_datetime("2018-01-01").date()) & 
        (df["Quote_date"] <= pd.to_datetime("2018-12-31").date())]


Adding TTM=0 row

In [ ]:
# Sort the dataframe by Quote_date and Expiry_date
df = df.sort_values(['Quote_date', 'Expiry_date'])

groups = df.groupby(['Expiry_date', 'Strike'])

for _, group in groups:

    # Sort group so that the last row is the one with the lowest TTM
    group = group.sort_values('TTM', ascending=False)

    # Taking row from option group (could be any) to be used in getting the Strike price
    last_row = group.iloc[-1]
    
    expiry_date = last_row['Expiry_date']
    # Get the underlying price on the day of expiry

    if expiry_date not in df_original['Quote_date'].values:
        print("Expiry date not in df_original: ", expiry_date)
        print("Weekday: ", expiry_date.weekday())
        # Delete group from df
        df = df.drop(group.index)
        continue

    underlying_last_on_expiry = df_original.loc[df_original['Quote_date'] == expiry_date, 'Underlying_last'].iloc[0]
    # Calculate the intrinsic value
    intrinsic_value = np.maximum(underlying_last_on_expiry - last_row['Strike'], 0)

    new_row = last_row.copy()
    new_row['Quote_date'] = expiry_date
    new_row['Expiry_date'] = expiry_date
    new_row['TTM'] = 0
    new_row['Underlying_last'] = underlying_last_on_expiry
    new_row['Price'] = intrinsic_value

    df = df.append(new_row, ignore_index=True)

# Sort the dataframe by Quote_date and Expiry_date
df = df.sort_values(['Quote_date', 'Expiry_date'])

### Functions

In [ ]:
def generate_buy_sell_signals(df, buy_threshold, sell_threshold):
    buy_signal = (df['Prediction'] / df['Price']) >= buy_threshold
    sell_signal = (df['Price'] / df['Prediction']) >= sell_threshold
    return buy_signal, sell_signal


def trader(df, buy_signal, sell_signal, starting_balance, price_cap_lower, ttm_cap_lower):
    df = df.copy()
    # Filter out options that expire before the last quote date
    last_date = df['Quote_date'].max()
    df = df[df['Expiry_date'] <= last_date]

    df = df[df["TTM"] >= ttm_cap_lower]
    
    df['Signal'] = 0
    df.loc[buy_signal, 'Signal'] = 1
    df.loc[sell_signal, 'Signal'] = -1
    df['Position_this_opt'] = 0
    df['Balance'] = starting_balance  # Initialize Balance column with starting_balance
    df['Profit'] = 0

    quote_date_grouped = df.groupby('Quote_date')

    balance = starting_balance

    for quote_date, group in quote_date_grouped:
        group = group.sample(frac=1)

        for _, row in group.iterrows():
            # Don't consider options with a price lower than 3
            if row['Price'] < price_cap_lower:
                continue

            option_position = df.loc[(df['Option_ID'] == row['Option_ID']) & (df['Quote_date'] <= row['Quote_date']), 'Position_this_opt'].iloc[-1]

            # Buy the option
            if row['Signal'] == 1 and balance >= row['Price'] and row["Quote_date"] != row["Expiry_date"]:
                balance -= row['Price']
                option_position += 1
                # Directly update values in the original dataframe
                df.loc[(df['Option_ID'] == row['Option_ID']) & (df['Quote_date'] == row['Quote_date']), 'Position_this_opt'] = option_position
                df.loc[(df['Option_ID'] == row['Option_ID']) & (df['Quote_date'] == row['Quote_date']), 'Balance'] = balance

            # Sell the option
            elif row['Signal'] == -1 and row["Quote_date"] != row["Expiry_date"] and balance > 0:
                balance += row['Price']
                option_position -= 1
                # Directly update values in the original dataframe
                df.loc[(df['Option_ID'] == row['Option_ID']) & (df['Quote_date'] == row['Quote_date']), 'Position_this_opt'] = option_position
                df.loc[(df['Option_ID'] == row['Option_ID']) & (df['Quote_date'] == row['Quote_date']), 'Balance'] = balance

            # Option expires
            elif row["Quote_date"] == row["Expiry_date"] and option_position != 0:
                intrinsic_value = max(0, row['Underlying_last'] - row['Strike'])
                adjustment = intrinsic_value if option_position >= 1 else -intrinsic_value
                balance += adjustment * abs(option_position)
                option_position = 0
                # Directly update values in the original dataframe
                df.loc[(df['Option_ID'] == row['Option_ID']) & (df['Quote_date'] == row['Quote_date']), 'Position_this_opt'] = option_position
                df.loc[(df['Option_ID'] == row['Option_ID']) & (df['Quote_date'] == row['Quote_date']), 'Balance'] = balance

            df.loc[(df['Quote_date'] == quote_date) & (df['Position_this_opt'] == 0), 'Balance'] = balance
        
        # Print date and balance if quote_date is 1. january, 1. april, 1. july or 1. october
        if quote_date.day == 1 or quote_date.day == 15:
            print("On, {}, the balance is: {}".format(quote_date, balance))
    return df


def calculate_options_value(df):
    df['Options_value'] = 0
    # Iterate over every Quote_date
    for date in df['Quote_date'].unique():
        # For every Quote date, multiply each row's Position_this_opt with the Price and sum it up.
        options_value_sum = (df.loc[df['Quote_date'] == date, 'Position_this_opt'] * df.loc[df['Quote_date'] == date, 'Price']).sum()
        # The result should be added to every row with the same Quote_date in the Options_value
        df.loc[df['Quote_date'] == date, 'Options_value'] = options_value_sum
    df['Total_value'] = df['Balance'] + df['Options_value']
    return df

### Run code

In [ ]:
def print_results(df, starting_balance):
    print("Starting balance", starting_balance)
    print("Ending balance", df['Balance'].iloc[-1])
    print("Number of trades", df['Signal'].abs().sum())
    profit = df['Balance'].iloc[-1] - starting_balance
    print("Profit", profit)
    print("Profit per trade", profit / df['Signal'].abs().sum())
    print("Profit per day", profit / (df['Quote_date'].max() - df['Quote_date'].min()).days)
    num_days = (df['Quote_date'].max() - df['Quote_date'].min()).days
    print("Number of days", num_days)
    print("Sharpe ratio: ", sharpe_ratio2(df))

In [ ]:
def sharpe_ratio(df):
    df = df.copy()
    # Group by Quote_date and keep the last row of each group
    df = df.groupby('Quote_date').last()
    # Calculate daily returns
    df['Daily_Returns'] = df['Total_value'].pct_change()
    # Convert the annual risk-free rate to a daily rate
    df['Daily_Rf'] = (1 + df['R'])**(1/252) - 1
    # Calculate the excess returns
    excess_returns = df['Daily_Returns'] - df['Daily_Rf']
    # Calculate sharpe ratio
    sharpe_ratio = np.sqrt(252) * (excess_returns.mean() / excess_returns.std())
    return sharpe_ratio

In [ ]:
def sharpe_ratio2(df, rf=0.02):
    df = df.copy()

    # Calculate daily returns
    df['Daily_Returns'] = df['Total_value'].pct_change()

    starting_balance = df['Balance'].iloc[0]
    ending_balance = df['Balance'].iloc[-1]

    num_days = (df['Quote_date'].max() - df['Quote_date'].min()).days

    avg_daily_return = (ending_balance / starting_balance)**(1/num_days) - 1

    avg_daily_return_excess = avg_daily_return - rf / 252

    df["Excess_Returns"] = df["Daily_Returns"] - rf / 252

    std_daily_excess_return = df["Excess_Returns"].std()

    sharpe_ratio = np.sqrt(252) * (avg_daily_return_excess / std_daily_excess_return)

    return sharpe_ratio

In [ ]:
if run_optimizing == False:
    df = df.copy()
    buy_signal, sell_signal = generate_buy_sell_signals(df, buy_threshold, sell_threshold)
    df = trader(df, buy_signal, sell_signal, starting_balance, price_cap_lower, ttm_cap_lower)
    df = calculate_options_value(df)
    print_results(df, starting_balance)

### Analysis

In [ ]:
def plot(df):
    # Just keep one row per Quote_date, and that should be the last row
    df = df.groupby('Quote_date').last()
    plt.plot(df['Total_value'], label='Total value')
    plt.plot(df['Balance'], label='Balance')
    plt.plot(df['Options_value'], label='Options value')
    plt.title('Portfolio value over time')
    # Make plot smaller
    ax = plt.gca()
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

In [ ]:
if run_optimizing == False:
    plot(df)

### Finding best thresholds

In [ ]:
if run_optimizing:
    # Initialize wandb
    !pip install wandb
    import wandb
    wandb.login(key="b47bcf387a0571c5520c58a13be35cda8ada0a99")

    # Define the hyperparameters
    sweep_config = {
    'method': 'grid',
    'name': 'Latest pricing model - All of 2019',
    'metric': {
        'goal': 'maximize', 
        'name': 'sharpe_ratio'
        },
    'parameters': {
        'buy_threshold': {
            'values': [1.01, 1.02, 1.04, 1.06, 1.1]},
        'sell_threshold': {
            'values': [1.01, 1.02, 1.04, 1.06, 1.1]},
        'starting_balance': {
            'values': [1000000]},
        'price_cap_lower': {
            'values': [0, 1, 3, 5]},
        'ttm_cap_lower': {
            'values': [0, 1, 3, 5]},
        }
    }
    sweep_id = wandb.sweep(sweep=sweep_config, project='options-trading') 


In [ ]:
if run_optimizing:
    def find_best_thresholds(df = df):
        run = wandb.init(project = "options-trading")
        df = df.copy()
        buy_signal, sell_signal = generate_buy_sell_signals(df, run.config.buy_threshold, run.config.sell_threshold)
        df = trader(df, buy_signal, sell_signal, run.config.starting_balance, price_cap_lower = run.config.price_cap_lower, ttm_cap_lower = run.config.ttm_cap_lower)
        df = calculate_options_value(df)
        run.log({"buy_threshold": run.config.buy_threshold, "sell_threshold": run.config.sell_threshold, "sharpe_ratio": sharpe_ratio2(df), "starting_balance": run.config.starting_balance, "ending_balance": df['Balance'].iloc[-1], "number_of_trades": df['Signal'].abs().sum()})
        # Wandb callback
        print("buy_threshold", run.config.buy_threshold)
        print("sell_threshold", run.config.sell_threshold)
        print("starting_balance", run.config.starting_balance)
        print("Ending balance", df['Balance'].iloc[-1])
        print("Number of trades", df['Signal'].abs().sum())
        print("sharpe_ratio", sharpe_ratio(df))
        plot(df)
        run.finish()

    wandb.agent(sweep_id=sweep_id, function=find_best_thresholds, project='options-trading', count = 1000)